# Sentiment Analysis on IMDB Movie Reviews with TF-IDF Text Embedding

## Loading Libraries

In [ ]:
!pip install git+https://github.com/troyhunterz/preprocess_tr.git

In [ ]:
import pandas as pd
import numpy as np

# scikit-learn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# custom library
import preprocess_tr as ps

# pickle
import pickle as pkl

# warnings
import warnings
warnings.filterwarnings('ignore')

## Loading Data

In [ ]:
df = pd.read_csv('dataset/imdb_reviews.txt', sep='\t', header=None)

## Text Preprocessing

In [ ]:
df.columns = ['reviews', 'sentiment']
df.head()

In [ ]:
df['reviews'] = df['reviews'].apply(lambda x: ps.contraction_to_expansion(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.rm_accented_chars(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.rm_html(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_emails(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_urls(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.rm_special_chars(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.lemmatize(x))
df['reviews'] = df['reviews'].apply(lambda x: str(x).lower())

In [ ]:
df.head(3)

## Data Preparation for Model Training

In [ ]:
X = df['reviews']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

X_train.shape, X_test.shape

## ML Model Building

In [ ]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(),
     'clf', LogisticRegression(solver='liblinear'))
])

In [ ]:
hyperparameters = {
    'tfidf__max_df': (.5, 1.),
    'tfidf__ngram_range': ((1,1),(1,2)),
    'tfidf__use_idf': (True, False),
    'tfifd__analyzer': ('word', 'char', 'char_wb'),

    'clf__penalty': ('l2', 'l1'),
    'clf__C': (1,2)
}

In [ ]:
clf = GridSearchCV(pipe, hyperparameters, n_jobs=-1, cv=None)

In [ ]:
%%time
clf.fit(X_train, X_test)

In [ ]:
print(f'{clf.best_score_}\n\n{clf.best_estimator_}')

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

## SVM Model

In [ ]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(),
     'clf', LinearSVC())
])

In [ ]:
hyperparameters = {
    'tfidf__max_df': (.5, 1.),
    'tfidf__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__analyzer': ('word', 'chars', 'char_wb'),

    'clf__C': (1,2, 2.5,3)
}

In [ ]:
clf = GridSearchCV(pipe, hyperparameters, n_jobs=-1, cv=None)

In [ ]:
%%time
clf.fit(X_train, y_train)

In [ ]:
clf.best_estimator_, clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

## Model Testing and Saving

In [ ]:
x = ['This is a great movie. I loved it', 'i have watched this movie. plot is straight. return my money']
clf.predict(x)

In [ ]:
pkl.dump(clf, open('model/model.pkl', 'wb'))